# PSTA

In [130]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import math
from multiprocessing import  Pool

In [95]:
df = pd.read_csv('../../Semesteroppgave/datasets/dataset/dataset_twitter/tweetsreplies4.tsv', encoding='cp1252', sep="\t")

In [131]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

## Preprocessing
Get detailed locations and timestamp and tokenize the text.
* Fetch more location info from longitude and latitude using reverse_encoder https://github.com/thampiman/reverse-geocoder
* Convert ms timestamps to datetime object.
* Preprocess text using the preprocessor https://github.com/s/preprocessor and remove stopwords using gensim.

In [96]:
coordinates = list(df[['latitude','longitude']].itertuples(index=False, name=None))
locations = rg.search(coordinates)

In [97]:
locations_df = pd.json_normalize(locations)[['name', 'admin1', 'admin2', 'cc']]

In [98]:
df['timestamp'] = pd.to_datetime(df['timestamp_ms'], unit='ms').dt.date

In [210]:
df_all = pd.concat([df, locations_df], axis=1)

In [212]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [213]:
# This takes a very long time, wonder if it hangs or something? It is the clean-command that is slow.
# It is always the same tweet that returns when I force stop it.
def add_features(df):
    df['text_'] = df['text'].apply(p.clean)
    return df
df_all = parallelize_dataframe(df_all.head(100), add_features)

In [214]:
# make lower case and remove white space, must be done after cleaning
df_all['doc']  = df_all['text_'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

In [203]:
# remove stopwords, must be done after cleaning and removal of white space
my_func = lambda x: remove_stopwords(x['doc'])
def remove_stopwords_f(df):
    df['doc'] = df['doc'].apply(remove_stopwords)
    return df
df_all['doc'] = parallelize_dataframe(df_all, remove_stopwords_f)

In [215]:
dataset = df_all[['doc', 'timestamp', 'admin1']]
dataset

,doc,timestamp,admin1
0,el bara lleva penaltis a favor y el madrid slo...,2015-10-03,Madrid
1,tu lo k debes hacer es callarte xk sois los ms...,2015-10-03,Castille-La Mancha
2,ya cambiara eso subnormal todo lo vuestro le l...,2015-10-03,Madrid
3,envidia no,2015-10-03,Andalusia
4,,2015-10-03,Madrid
...,...,...,...
95,seguime por favoor,2015-10-03,Buenos Aires
96,hermooosos los amoo,2015-10-03,Buenos Aires F.D.
97,son hermosos,2015-10-03,Santiago del Estero
98,me seguiss nicoo t,2015-10-03,Buenos Aires


In [205]:
dataset_agg = dataset.groupby(['timestamp', 'admin1']).agg(list)

In [206]:
# define vocabulary V
results = set()
df_all['doc'].str.split().apply(results.update)
results

AttributeError: Can only use .str accessor with string values!

In [184]:
df_all['doc']

0     el bar lleva penaltis favor y el madrid slo va...
1     as_tomasroncero tu lo k debes hacer es callart...
2     as_tomasroncero ya cambiara eso subnormal todo...
3                               as_tomasroncero envidia
4                     as_tomasroncero http t muugc3nre3
                            ...                        
95                   nicolasocchiato seguime por favoor
96                  nicolasocchiato hermooosos los amoo
97                         nicolasocchiato son hermosos
98                      nicolasocchiato seguiss nicoo t
99    nicolasocchiato que buen look te juro que encanto
Name: doc, Length: 100, dtype: object